<a href="https://colab.research.google.com/github/wmd20/SL_plant-disease-detection/blob/master/%E7%BB%9F%E8%AE%A1%E5%A4%A7%E4%BD%9C%E4%B8%9A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/wmd20/SL_plant-disease-detection.git

Cloning into 'SL_plant-disease-detection'...
remote: Enumerating objects: 31581, done.
remote: Total 31581 (delta 0), reused 0 (delta 0), pack-reused 31581
Receiving objects: 100% (31581/31581), 2.81 GiB | 47.05 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Checking out files: 100% (31741/31741), done.


In [2]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import time
import random
import matplotlib.pyplot as plt
def readFile(path,flag):
    image_dir=sorted(os.listdir(path))
    x=np.zeros((len(image_dir),256,256,3),dtype=np.uint8)
    y=np.zeros((len(image_dir)),dtype=np.uint8)
    for i,file in enumerate(image_dir):
        img=cv2.imread(os.path.join(path,file))
        x[i,:,:]=cv2.resize(img,(256,256))
        if flag:
            y[i]=int(file.split(',')[0]) # label 的映射需要重新定义
    if flag:
        return x,y
    else:
        return x

# 分别将 training set,validation set,testing set 用readFile函数读进来
workSpace_dir='SL_plant-disease-detection'
print("Reading data...")
train_x,train_y=readFile(os.path.join(workSpace_dir,"trainingset_part1"),True)
print("Size of training data={}".format(len(train_x)))
val_x,val_y=readFile(os.path.join(workSpace_dir,"trainingset_part1"),True)
print("Size of validation data={}".format(len(val_x)))

Reading data...
Size of training data=2981
Size of validation data=2981


In [43]:
#training 時做 data augmentation
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(), #隨機將圖片水平翻轉
    transforms.RandomRotation(15), #隨機旋轉圖片
    transforms.ToTensor(), #將圖片轉成 Tensor，並把數值normalize到[0,1](data normalization)
])
#testing 時不需做 data augmentation
test_transform = transforms.Compose([
    transforms.ToPILImage(),                                    
    transforms.ToTensor(),
])
class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = x
        # label is required to be a LongTensor
        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y)
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            Y = self.y[index]
            return X, Y
        else:
            return X
batch_size=128
train_set=ImgDataset(train_x,train_y,train_transform)
val_set=ImgDataset(val_x,val_y,test_transform)
train_loader=DataLoader(train_set,batch_size=batch_size,shuffle=True)
val_loader=DataLoader(val_set,batch_size=batch_size,shuffle=False)

In [64]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import time
import random
import matplotlib.pyplot as plt
torch.cuda.empty_cache()

class Residual_Block(nn.Module):
    def __init__(self, i_channel, o_channel, stride, down_sample = None):
        super(Residual_Block, self).__init__()
        # 第一个卷积层，卷积块大小为 1 * 1
        self.conv1 = nn.Conv2d(in_channels = i_channel,
                    out_channels = o_channel,
                    stride = stride,
                    padding = 0,
                    kernel_size = 1,
                    bias = False)
        # BatchNorm2d()对小批量3d数据组成的4d输入进行批标准化操作
        # 主要为了防止神经网络退化
        self.bn1 = nn.BatchNorm2d(o_channel)
        # Relu
        self.relu1 = nn.ReLU(inplace = True)
        # 第二卷积层, 3 * 3 的卷积
        self.conv2 = nn.Conv2d(in_channels = o_channel, 
                    out_channels = o_channel, 
                    kernel_size = 3, 
                    stride = 1, 
                    padding = 1,
                    bias = False)
        self.bn2 = nn.BatchNorm2d(o_channel)
        # Relu
        self.relu2 = nn.ReLU(inplace = True)
        # 第三卷积层，1 * 1的卷积块，输出通道为 4 * 第一个卷积输出通道
        self.conv3 = nn.Conv2d(in_channels = o_channel,
                    out_channels = 4 * o_channel,
                    kernel_size = 1,
                    stride = 1,
                    padding = 0,
                    bias = False)
        # bn
        self.bn3 = nn.BatchNorm2d(4 * o_channel)
        # relu
        self.relu3 = nn.ReLU(inplace = True)
        self.down_sample = down_sample
        self.stride = stride
        
    # 前向传播
    def forward(self, x):
        residual = x
        print('residual.shape = ' + str(residual.shape))
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        #print('第一层卷积结束后的尺寸' + str(out.shape))
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)
        #print('第二层卷积结束后的尺寸' + str(out.shape))
        out = self.conv3(out)
        out = self.bn3(out)
        #print('out.shape = ' + str(out.shape))
        if self.stride == 2:
            down_sample = nn.Sequential(nn.Conv2d(residual.size(1),
                                residual.size(1) * 2,
                                kernel_size = 1,
                                stride = 2,
                                padding = 0,
                                bias = False),
                                nn.BatchNorm2d(residual.size(1) * 2))
            residual = down_sample(residual)
        #if self.down_sample:
           # residual = self.down_sample(x) # 下采样
        # 将单元的输入直接与单元输出加在一起
        if residual.shape != out.shape:
            residual = torch.cat((residual.cuda(), torch.zeros(x.size(0), out.size(1) - x.size(1), out.size(2), out.size(3).cuda())), 1).cuda()
        #print('residual.size = ' + str(residual.shape)) 
        #print()
        out += residual
        out = self.relu3(out)
        #print(out.shape)
        return out
    
# 定义残差神经网络
class ResNet50(nn.Module):
    # 残差块、层数、类别数量
    def __init__(self, num_classes):
        super(ResNet50, self).__init__()
        # 第一层卷积， 输入3通道， 输出64通道， 7 * 7的卷积
        self.conv1 = nn.Conv2d(in_channels = 3, 
                    out_channels = 64, 
                    kernel_size = 7,  
                    stride = 2,
                    padding = 0,
                    bias = False)
        # 标记输出通道，为后面输入输出相加时是否需要做维度变换提供判断，
        self.in_channels = 64
        # 归一化
        self.bn1 = nn.BatchNorm2d(64)
        # Relu
        self.relu1 = nn.ReLU(inplace = True)
        # 最大池化，3*3的池化窗口，stride = 2
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2)
        # 第一种残差块
        self.block1 = Residual_Block(i_channel = 64, o_channel = 64, stride = 1)
        self.block2 = Residual_Block(i_channel = 256, o_channel = 64, stride = 1)
        self.block3 = Residual_Block(i_channel = 256, o_channel = 64, stride = 1)
        
        # 第二类残差块
        self.block4 = Residual_Block(i_channel = 256, o_channel = 128, stride = 2)
        self.block5 = Residual_Block(i_channel = 512, o_channel = 128, stride = 1)
        self.block6 = Residual_Block(i_channel = 512, o_channel = 128, stride = 1)
        self.block7 = Residual_Block(i_channel = 512, o_channel = 128, stride = 1)
        
        # 第三类残差块
        self.block8 = Residual_Block(i_channel = 512, o_channel = 256, stride = 2)
        self.block9 = Residual_Block(i_channel = 1024, o_channel = 256, stride = 1)
        self.block10 = Residual_Block(i_channel = 1024, o_channel = 256, stride = 1)
        self.block11 = Residual_Block(i_channel = 1024, o_channel = 256, stride = 1)
        self.block12 = Residual_Block(i_channel = 1024, o_channel = 256, stride = 1)
        self.block13 = Residual_Block(i_channel = 1024, o_channel = 256, stride = 1)
        
        # 第四类残差块
        self.block14 = Residual_Block(i_channel = 1024, o_channel = 512, stride = 2)
        self.block15 = Residual_Block(i_channel = 2048, o_channel = 512, stride = 1)
        self.block16 = Residual_Block(i_channel = 2048, o_channel = 512, stride = 1)
        # 平均池化层，窗口大小 8 * 8
        self.avg_pool = nn.AvgPool2d(8)
        # 全连接层
        self.fc = nn.Linear(2048, num_classes)
        # softmax层
        self.softmax = nn.Softmax(dim = 1)
        
    # 前向传播
    def forward(self, x):
      print('前向传播' + str(x.shape))
      out = self.conv1(x)
      print('第一层卷积' + str(out.shape))
      out = self.bn1(out)
      out = self.relu1(out)
      #print('第一层卷积' + str(out.shape))
      out = self.maxpool(out)
      #print('最大池化后' + str(out.shape))
      out = self.block1(out)
      #print('block1后' + str(out.shape))
      out = self.block2(out)
      #print('block2后' + str(out.shape))
      out = self.block3(out)
      #print('第一残差块之后' + str(out.shape))
      out = self.block4(out)
      #print('block4后' + str(out.shape))
      out = self.block5(out)
      out = self.block6(out)
      out = self.block7(out)
      #print('第二类残差块之后：' + str(out.shape))
      out = self.block8(out)
      out = self.block9(out)
      out = self.block10(out)
      out = self.block11(out)
      out = self.block12(out)
      out = self.block13(out)
      #print('第三类残差块之后' + str(out.shape))
      out = self.block14(out)
      out = self.block15(out)
      out = self.block16(out)
      #print('第四类残差块之后' + str(out.shape))
      out = self.avg_pool(out)
      #将四维张量转换为二维张量之后，才能作为全连接层的输入
      out = out.view(out.size()[0], -1)
      out = self.fc(out)
      #print('全连接层输出' + str(out))
      out = self.softmax(out)
      return out
    
#更新学习率
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        
# 固定随机种子
random.seed(1)
# 超参数设定
batch_size = 16
learning_rate = 1e-3
# 定义模型
model = ResNet50(8).cuda()
# 因为是分类任务，所以使用交叉熵损失 
loss=nn.CrossEntropyLoss() # 因为是 classification task,所以loss使用crossEntropyLoss
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
num_epoch=30

for epoch in range(num_epoch):
    epoch_start_time=time.time()
    train_acc=0.0
    train_loss=0.0
    val_acc=0.0
    val_loss=0.0
    
    model.train()
    
    for i,data in enumerate(train_loader):
      print('data[0].shape' + str(data[0].shape))
      optimizer.zero_grad() # 用optimizer 将model参数的gradient归零
      train_pred=model(data[0].cuda()) #利用model得到预测的概率分布，这边实际上就是去call model的froward 函数
      batch_loss=loss(train_pred,data[1].cuda()) #计算loss 注意 prediction跟label必须同时待在CPU或是GPU上
      batch_loss.backward() # 利用back probagation 算出每个参数的gradient
      optimizer.step() #更新参数
      
      train_acc+=np.sum(np.argmax(train_pred.data.numpy(),axis=1)==data[1].numpy())
      train_loss+=batch_loss.item()
        
    model.eval()
    with torch.no_grad(): # torch.no_grad() 是一个上下文管理器，被该语句 wrap 起来的部分将不会track 梯度
        for i,data in enumerate(val_loader):
          val_pred=model(data[0].cuda())
          batch_loss=loss(val_pred,data[1].cuda())
          
          val_acc+=np.sum(np.argmax(val_pred.data.numpy(),axis=1)==data[1].numpy)
          val_loss+=batch_loss.item()
        #将结果 print 出来
        print('[%03d/%03d] %2.2f sec(s) Trian Acc:%3.6f Loss:%3.6f | Val Acc:%3.6f Loss:%3.6f' %\
             (epoch+1,num_epoch,time.time()-epoch_start_time,\
             train_acc/train_set.__len__(),train_loss/train_set.__len__(),\
             val_acc/val_set.__len__(),val_loss/val_set.__len__()))

data[0].shapetorch.Size([128, 3, 256, 256])
前向传播torch.Size([128, 3, 256, 256])
第一层卷积torch.Size([128, 64, 125, 125])
residual.shape = torch.Size([128, 64, 62, 62])


AttributeError: ignored